In [1]:
class trigger_rates:
    '''
    This class estimates the average single DOM rate (sdr) depending on the PMT rate seen by a PMT in a DOM.
    The sdr depends on the number of PMT hits on a DOM.
    
    k: number of PMT hits needed for triggering
    rates: rates of the PMT
    time_fraction: fraction of time at a particular rate

    The singleDOMhalf function considers half correlation in a DOM, where one hals sees the bio, and the other
    one sees the k40 rate.
    '''
    def __init__(self,k,rates,time_fraction,pmts=16,correlated = True):
        self.k             = k
        self.rates         = rates
        self.time_fraction = time_fraction
        self.pmts          = pmts
        self.correlated    = correlated
   
    def singleDOMrate(self):        
        n = len(self.rates)
        comb_value = math.comb(self.pmts, self.k)
        if self.correlated == True:            
            averages = [((self.rates[i])**self.k * 10e-9 **(self.k-1)* self.time_fraction[i] * comb_value) for i in range(n)]
            
            if self.k == 2:
                sdr = [sum(averages[:i+1])+1000 for i in range(n)]
                
            else:
                sdr = [sum(averages[:i+1]) for i in range(n)]
        elif self.correlated == False:
            cor = [sum(self.rates[:i+1])*self.time_fraction[i] for i in range(n)]
            averages = [((self.rates[i]*cor[i]**(self.k-1)) * 10e-9**(self.k-1) * self.time_fraction[i] * comb_value) for i in range(n)]
            sdr = [sum(averages[:i+1]+1000) for i in range(n)]
        rates = [i*1e-3 for i in self.rates]
        return rates, sdr
    
    def singleDOMhalf(self):
        k40 = 10000
        dr = []
        sdr = []
        n = len(self.rates)
        for i in range(n):
            f = 0
            for j in range(self.k+1):                    
                f += (self.rates[i])**(self.k-j) * math.comb(8,self.k-j) * k40**(j) * math.comb(8,j)* 10e-9**(self.k-1) * self.time_fraction[i]
            dr.append(f)
            if self.k == 2:
                sdr.append(sum(dr[:i+1])+1000)
            else:
                sdr.append(sum(dr[:i+1]))
        rates = [i*1e-3 for i in self.rates]
        return rates,sdr

    def singleDOMnominal(self,rate):
        k40 = 10000
        dr = []
        sdr = []
        f = 0
        for j in range(self.k+1):                    
            f += (rate)**(self.k-j) * math.comb(8,self.k-j) * k40**(j) * math.comb(8,j)* 10e-9**(self.k-1)
            if self.k == 2:
                return f + 1000
            else:
                return f
    